In [6]:
import tensorflow as tf
import matplotlib.pyplot as plt
import time

# Parameters
learning_rate = 0.0001 #0.001
training_epochs = 120 #15
batch_size = 100
display_step = 1

In [2]:
import csv
import os
import numpy as np

#인코딩중요
#라벨만들기
with open('./trainSet.csv', 'r', encoding='UTF-8-sig') as raw:
    lines = raw.readlines()
data = list(csv.reader(lines))
data = sorted(data, key=lambda x: x[0])

#이름빼기
_Y = [y[1] for y in data]

_Y = np.array(_Y)
_Y = _Y.astype(float) # shape(-1)
_Y = [[1-y, y] for y in _Y] # shape(-1, 2)


#데이터만들기
with open('./features4096.csv', 'r', encoding='UTF-8-sig') as raw:
    lines = raw.readlines()
data = list(csv.reader(lines))
data = sorted(data, key=lambda x: x[0])

#이름빼기
_X = [x[1:] for x in data]

_X = np.array(_X)
_X = _X.astype(float) # shape(-1, 1024)


all_set = tuple(zip(_X, _Y)) # shape(10000, 2, ~)
M_all_set = [all_set[i] for i in range(len(all_set)) if all_set[i][1][1] == 1] # 7000
N_all_set = [all_set[i] for i in range(len(all_set)) if all_set[i][1][0] == 1] # 3000
np.random.shuffle(M_all_set)
np.random.shuffle(N_all_set)

# 악성,정상 개수 맞추기
if len(M_all_set) > len(N_all_set):
    M_all_set = M_all_set[:len(N_all_set)]
else:
    N_all_set = N_all_set[:len(M_all_set)]

all_set = M_all_set + N_all_set
np.random.shuffle(all_set) # 6000

train_set = all_set[len(all_set)//5:] # 4800
test_set = all_set[:len(all_set)//5] # 1200

train_X = np.array([X for X,Y in train_set])
train_Y = np.array([Y for X,Y in train_set])
test_X = np.array([X for X,Y in test_set])
test_Y = np.array([Y for X,Y in test_set])


M_test_X = [X for X,Y in test_set if Y[1] == 1] # label의 악성이면
M_test_Y = [Y for X,Y in test_set if Y[1] == 1] # label의 악성이면

N_test_X = [X for X,Y in test_set if Y[0] == 1] # label의 정상이면
N_test_Y = [Y for X,Y in test_set if Y[0] == 1] # label의 정상이면



In [3]:
with tf.device('/gpu:0'):
    X = tf.placeholder(tf.float32, [None, 4096])
    Y = tf.placeholder(tf.float32, [None, 2])
    keep_prob = tf.placeholder(tf.float32)

    #X = tf.nn.dropout(X, keep_prob=keep_prob)

    # Layer1
    W1 = tf.get_variable('%f'%(np.random.random()), shape=[4096,2000], initializer=tf.contrib.layers.xavier_initializer())
    b1 = tf.Variable(tf.random_normal([2000]))
    L1 = tf.nn.relu(tf.matmul(X, W1) + b1)
    L1 = tf.nn.dropout(L1, keep_prob=keep_prob)

    # Layer2
    W2 = tf.get_variable('%f'%(np.random.random()), shape=[2000,1000], initializer=tf.contrib.layers.xavier_initializer())
    b2 = tf.Variable(tf.random_normal([1000]))
    L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)
    L2 = tf.nn.dropout(L2, keep_prob=keep_prob)

    # Layer3
    W3 = tf.get_variable('%f'%(np.random.random()), shape=[1000,500], initializer=tf.contrib.layers.xavier_initializer())
    b3 = tf.Variable(tf.random_normal([500]))
    L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)
    L3 = tf.nn.dropout(L3, keep_prob=keep_prob)

    # Layer4
    W4 = tf.get_variable('%f'%(np.random.random()), shape=[500,250], initializer=tf.contrib.layers.xavier_initializer())
    b4 = tf.Variable(tf.random_normal([250]))
    L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)
    L4 = tf.nn.dropout(L4, keep_prob=keep_prob)

    # Layer5
    W5 = tf.get_variable('%f'%(np.random.random()), shape=[250,100], initializer=tf.contrib.layers.xavier_initializer())
    b5 = tf.Variable(tf.random_normal([100]))
    L5 = tf.nn.relu(tf.matmul(L4, W5) + b5)
    L5 = tf.nn.dropout(L5, keep_prob=keep_prob)
    
    # Layer6
    W6 = tf.get_variable('%f'%(np.random.random()), shape=[100,2], initializer=tf.contrib.layers.xavier_initializer())
    b6 = tf.Variable(tf.random_normal([2]))
    hypothesis = tf.matmul(L5, W6) + b6


    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis, labels=Y))
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [7]:
# Training
sess = tf.Session()
sess.run(tf.global_variables_initializer())
#sess.run(tf.initialize_all_variables())

# Training cycle
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(len(train_X) / batch_size)

    for i in range(total_batch):
        batch_xs, batch_ys = train_X[i*100:(i+1)*100], train_Y[i*100:(i+1)*100]
        feed_dict = {X: batch_xs, Y: batch_ys, keep_prob: 0.7}
        #feed_dict = {X: batch_xs, Y: batch_ys}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        
        avg_cost += c / total_batch #뭉치갯수로 나누기
        #time.sleep(0.03)
        
    if (epoch+1)%5 == 0:
        print('Epoch : ', '%04d' % (epoch + 1), 'cost = ', '{:.9f}'.format(avg_cost))


Epoch :  0005 cost =  0.578397397
Epoch :  0010 cost =  0.398824014
Epoch :  0015 cost =  0.288723690
Epoch :  0020 cost =  0.274288728
Epoch :  0025 cost =  0.228247695
Epoch :  0030 cost =  0.176345478
Epoch :  0035 cost =  0.156589655
Epoch :  0040 cost =  0.157356316
Epoch :  0045 cost =  0.126529413
Epoch :  0050 cost =  0.108971750
Epoch :  0055 cost =  0.104758474
Epoch :  0060 cost =  0.114326274
Epoch :  0065 cost =  0.095923922
Epoch :  0070 cost =  0.122611254
Epoch :  0075 cost =  0.101449987
Epoch :  0080 cost =  0.081010613
Epoch :  0085 cost =  0.067791111
Epoch :  0090 cost =  0.081094576
Epoch :  0095 cost =  0.096617021
Epoch :  0100 cost =  0.078173277
Epoch :  0105 cost =  0.056941681
Epoch :  0110 cost =  0.064067221
Epoch :  0115 cost =  0.061620003
Epoch :  0120 cost =  0.049337344


In [8]:
# TEST
is_correct = tf.equal(tf.arg_max(hypothesis, 1), tf.arg_max(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

feed_dict = {X: test_X, Y: test_Y, keep_prob: 1}
print("Accuracy : ", accuracy.eval(session=sess, feed_dict=feed_dict))

feed_dict = {X: M_test_X, Y: M_test_Y, keep_prob: 1}
print("정탐율 : ", accuracy.eval(session=sess, feed_dict=feed_dict))

feed_dict = {X: N_test_X, Y: N_test_Y, keep_prob: 1}
print("오탐율 : ", accuracy.eval(session=sess, feed_dict=feed_dict))


Accuracy :  0.9241667
정탐율 :  0.8956661
오탐율 :  0.95493937
